In [2]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [4]:
columns = ["age", "workclass", "fnlwgt", "education", "education-num", "martial-status",
        "occupation", "relationship", "race", "sex", "capital-gain", "capital-loss",
        "hours-per-week", "native-country", "income"]
data = pd.read_csv('data/adult/adult.data', names=columns)

In [5]:
data.head()

,age,workclass,fnlwgt,education,education-num,martial-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


### Pre-processing

In [6]:
label_encoder = LabelEncoder()
data['income'] = label_encoder.fit_transform(data['income'])

# Convert categorical variables to one-hot encoding
data = pd.get_dummies(data, drop_first=True, dtype=float)

data.head()

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week,income,workclass_ Federal-gov,workclass_ Local-gov,workclass_ Never-worked,...,native-country_ Portugal,native-country_ Puerto-Rico,native-country_ Scotland,native-country_ South,native-country_ Taiwan,native-country_ Thailand,native-country_ Trinadad&Tobago,native-country_ United-States,native-country_ Vietnam,native-country_ Yugoslavia
0,39,77516,13,2174,0,40,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,50,83311,13,0,0,13,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,38,215646,9,0,0,40,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,53,234721,7,0,0,40,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,28,338409,13,0,0,40,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
X = data.drop('income', axis=1).values
y = data['income'].values

print(data.columns.tolist())

['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week', 'income', 'workclass_ Federal-gov', 'workclass_ Local-gov', 'workclass_ Never-worked', 'workclass_ Private', 'workclass_ Self-emp-inc', 'workclass_ Self-emp-not-inc', 'workclass_ State-gov', 'workclass_ Without-pay', 'education_ 11th', 'education_ 12th', 'education_ 1st-4th', 'education_ 5th-6th', 'education_ 7th-8th', 'education_ 9th', 'education_ Assoc-acdm', 'education_ Assoc-voc', 'education_ Bachelors', 'education_ Doctorate', 'education_ HS-grad', 'education_ Masters', 'education_ Preschool', 'education_ Prof-school', 'education_ Some-college', 'martial-status_ Married-AF-spouse', 'martial-status_ Married-civ-spouse', 'martial-status_ Married-spouse-absent', 'martial-status_ Never-married', 'martial-status_ Separated', 'martial-status_ Widowed', 'occupation_ Adm-clerical', 'occupation_ Armed-Forces', 'occupation_ Craft-repair', 'occupation_ Exec-managerial', 'occupation_ Farming-fishing', 'occupa

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
X_train.shape

(26048, 100)

### Model Training

In [10]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

In [11]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)

model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.1, callbacks=[early_stopping])

Epoch 1/100
733/733 ━━━━━━━━━━━━━━━━━━━━ 1s 567us/step - accuracy: 0.6565 - loss: 406.6238 - val_accuracy: 0.5812 - val_loss: 12.4031
Epoch 2/100
733/733 ━━━━━━━━━━━━━━━━━━━━ 0s 480us/step - accuracy: 0.6868 - loss: 76.6348 - val_accuracy: 0.8050 - val_loss: 37.5094
Epoch 3/100
733/733 ━━━━━━━━━━━━━━━━━━━━ 0s 475us/step - accuracy: 0.6846 - loss: 45.5701 - val_accuracy: 0.7482 - val_loss: 3.7450
Epoch 4/100
733/733 ━━━━━━━━━━━━━━━━━━━━ 0s 471us/step - accuracy: 0.6770 - loss: 17.3992 - val_accuracy: 0.7942 - val_loss: 1.9683
Epoch 5/100
733/733 ━━━━━━━━━━━━━━━━━━━━ 0s 458us/step - accuracy: 0.6755 - loss: 12.8806 - val_accuracy: 0.3317 - val_loss: 1.0638
Epoch 6/100
733/733 ━━━━━━━━━━━━━━━━━━━━ 0s 458us/step - accuracy: 0.7031 - loss: 1.5609 - val_accuracy: 0.8058 - val_loss: 0.7713
Epoch 7/100
733/733 ━━━━━━━━━━━━━━━━━━━━ 0s 461us/step - accuracy: 0.7157 - loss: 0.9787 - val_accuracy: 0.2795 - val_loss: 1.4079
Epoch 8/100
733/733 ━━━━━━━━━━━━━━━━━━━━ 0s 455us/step - accuracy: 0.7444 -

In [12]:
loss, accuracy = model.evaluate(X_test, y_test)

204/204 ━━━━━━━━━━━━━━━━━━━━ 0s 281us/step - accuracy: 0.7601 - loss: 0.5494


In [13]:
y_pred = model.predict(X_test)

204/204 ━━━━━━━━━━━━━━━━━━━━ 0s 340us/step


## Model Card Generation

In [14]:
from patra_model_card.patra_model_card import ModelCard, AIModel, BiasAnalysis, ExplainabilityAnalysis

/Users/neeleshkarthikeyan/d2i/patra-toolkit/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [15]:
mc = ModelCard(
            name="UCI Adult Data Analysis model using Tensorflow",
            version="0.1",
            short_description="UCI Adult Data analysis using Tensorflow for demonstration of Patra Model Cards.",
            full_description="We have trained a ML model using the tensorflow framework to predict income for the UCI Adult Dataset. We leverage this data to run the Patra model cards to capture metadata about the model as well as fairness and explainability metrics.",
            keywords="uci adult, tensorflow, explainability, fairness, patra",
            author="Sachith Withana",
            input_type="Tabular",
            category="classification",
            foundational_model="None"
        )

mc.input_data = 'https://archive.ics.uci.edu/dataset/2/adult'
mc.output_data = 'https://github.iu.edu/d2i/dockerhub/tensorflow/adult_modelv01'

In [16]:
ai_model = AIModel(
            name="Income prediction tensorflow model",
            version="0.1",
            description="Census classification problem using Tensorflow Neural Network using the UCI Adult Dataset",
            owner="Sachith Withana",
            location="https://github.iu.edu/d2i/uci_adult/tensorflow_model",
            license="BSD-3 Clause",
            framework="tensorflow",
            model_type="dnn",
            test_accuracy=accuracy
        )
ai_model.populate_model_structure(model)

In [17]:
ai_model.add_metric("Test loss", loss)
ai_model.add_metric("Epochs", 100)
ai_model.add_metric("Batch Size", 32)
ai_model.add_metric("Optimizer", "Adam")
ai_model.add_metric("Learning Rate", 0.0001)
ai_model.add_metric("Input Shape", "(26048, 100)")

In [18]:
mc.ai_model = ai_model

In [19]:
mc.populate_requirements()

In [20]:
print(mc)

{
    "name": "UCI Adult Data Analysis model using Tensorflow",
    "version": "0.1",
    "short_description": "UCI Adult Data analysis using Tensorflow for demonstration of Patra Model Cards.",
    "full_description": "We have trained a ML model using the tensorflow framework to predict income for the UCI Adult Dataset. We leverage this data to run the Patra model cards to capture metadata about the model as well as fairness and explainability metrics.",
    "keywords": "uci adult, tensorflow, explainability, fairness, patra",
    "author": "Sachith Withana",
    "input_type": "Tabular",
    "category": "classification",
    "input_data": "https://archive.ics.uci.edu/dataset/2/adult",
    "output_data": "https://github.iu.edu/d2i/dockerhub/tensorflow/adult_modelv01",
    "foundational_model": "None",
    "ai_model": {
        "name": "Income prediction tensorflow model",
        "version": "0.1",
        "description": "Census classification problem using Tensorflow Neural Network usi

In [21]:
y_pred = model.predict(X_test)
y_pred = (y_pred >= 0.5).flatten()
mc.populate_bias(X_test, y_test, y_pred, "gender", X_test[:, 58], model)

204/204 ━━━━━━━━━━━━━━━━━━━━ 0s 560us/step


In [22]:
x_columns = data.columns.tolist()
x_columns.remove('income')

mc.populate_xai(X_test[:10], x_columns, model, 10)

In [23]:
print(mc.bias_analysis)

{'demographic_parity_diff': 0.006310121347580221, 'equal_odds_difference': 0.006941370439513206}


In [24]:
print(mc.xai_analysis)

{'capital_gain': 2.3179584085486482e-10, 'workclass__Self_emp_not_inc': 1.9868214943633925e-10, 'education__Bachelors': 1.3245476629089284e-10, 'martial_status__Married_civ_spouse': 1.3245476613669518e-10, 'occupation__Adm_clerical': 1.1589792042743241e-10, 'martial_status__Never_married': 9.934107471816962e-11, 'fnlwgt': 9.934107456397199e-11, 'occupation__Machine_op_inspct': 6.622738329964407e-11, 'race__Other': 6.622738329964407e-11, 'race__White': 6.622738314544642e-11}


In [25]:
print(mc)

{
    "name": "UCI Adult Data Analysis model using Tensorflow",
    "version": "0.1",
    "short_description": "UCI Adult Data analysis using Tensorflow for demonstration of Patra Model Cards.",
    "full_description": "We have trained a ML model using the tensorflow framework to predict income for the UCI Adult Dataset. We leverage this data to run the Patra model cards to capture metadata about the model as well as fairness and explainability metrics.",
    "keywords": "uci adult, tensorflow, explainability, fairness, patra",
    "author": "Sachith Withana",
    "input_type": "Tabular",
    "category": "classification",
    "input_data": "https://archive.ics.uci.edu/dataset/2/adult",
    "output_data": "https://github.iu.edu/d2i/dockerhub/tensorflow/adult_modelv01",
    "foundational_model": "None",
    "ai_model": {
        "name": "Income prediction tensorflow model",
        "version": "0.1",
        "description": "Census classification problem using Tensorflow Neural Network usi

In [26]:
mc.validate()

True

In [28]:
mc.submit("http://149.165.153.142:5002")

{'message': 'Successfully uploaded the model card',
 'model_card_id': '44a9621b334c753d2c581efdccc29bc2b81148d846d8a9fd8bdcebd4d3856f60'}

In [29]:
mc.save("../model_cards/tensorflow_adult_nn_MC.json")